# Vegetation Relevés 2025 — Exploratory Diversity & Spatial Analysis (Notebook)

Notebook step-by-step (senza funzioni wrapper) per analizzare **Vegetation relevés 2025.xlsx**.
Ogni cella produce output immediato così puoi intervenire e modificare.

**Pipeline**: caricamento → normalizzazione colonne → statistiche → griglia (ricchezza/Shannon/Evenness) →
hotspot → DBSCAN (metri) → indici di diversità → pattern temporali (se disponibili) → salvataggi e mini-report.


In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import os
print(os.getcwd())
os.chdir("..")

c:\Users\ares\Desktop\Andrea\Leiden\Leiden University\Semestre 3\Statistical Consulting\Assignment\Submission code\Scientific


In [2]:
print(os.getcwd())

c:\Users\ares\Desktop\Andrea\Leiden\Leiden University\Semestre 3\Statistical Consulting\Assignment\Submission code


## 1) Configurazione e caricamento dati

In [3]:
filepath='Data/Scientific Data/Vegetation_relevés_2025.xlsx'
df=pd.read_excel(filepath)
df.head()

,Nr. in table,Unnamed: 1,Unnamed: 2,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,100,SPECIES_NR,Unnamed: 104
0,Plot,NaN,NaN,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,100,NaN,NaN
1,Plot observation,NaN,NaN,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,100,NaN,NaN
2,Opnamenummer,NaN,NaN,1,2,3,4,5,6,7,...,93,94,95,96,97,98,99,100,NaN,NaN
3,Associa (1),NaN,NaN,r14RG04,r16RG20,r16RG20,r32RG01,r32RG01,r32RG01,r16RG20,...,r18AB01,r32RG02,r08RG12,r08BA02,r16RG10,r08RG13,r42RG05,r40AA02,NaN,NaN
4,Bedekkingsschaal,NaN,NaN,"Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)",...,"Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)",NaN,NaN


In [4]:
Data=df[:][37:]
for i in range(2, Data.shape[1]):
    Data.columns.values[i] = f"Area {i-1}"
Data.columns.values[0] = "Species"
Data.columns.values[1] = "Name"
Data.head() 


,Species,Name,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,...,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102,Area 103
37,Hogere plant,Achillea millefolium - Duizendblad,NaN,8,NaN,2,38,18,2,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Duizendblad
38,Hogere plant,Aira caryophyllea - Zilverhaver,NaN,38,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,Zilverhaver
39,Hogere plant,Aira praecox - Vroege haver,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,Vroege haver
40,Hogere plant,Aphanes australis - Kleine leeuwenklauw,NaN,18,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,Kleine leeuwenklauw
41,Hogere plant,Arenaria serpyllifolia - Gewone zandmuur,NaN,38,3,NaN,NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,Gewone zandmuur


### Normalizzazione nomi colonna

In [5]:
# #RUN this cell only once
# ################################
# ################################
# ################################
# ################################
Data.head()
Data = Data.drop(columns=["Area 1"], errors="ignore")
Data = Data.drop(columns=["Area 1"], errors="ignore")

In [6]:
Data.head()
for i in range(2, Data.shape[1]):
    Data.columns.values[i] = f"Area {i-1}"
Data.head()

,Species,Name,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,...,Area 93,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102
37,Hogere plant,Achillea millefolium - Duizendblad,8,NaN,2,38,18,2,8,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Duizendblad
38,Hogere plant,Aira caryophyllea - Zilverhaver,38,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,Zilverhaver
39,Hogere plant,Aira praecox - Vroege haver,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,Vroege haver
40,Hogere plant,Aphanes australis - Kleine leeuwenklauw,18,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,Kleine leeuwenklauw
41,Hogere plant,Arenaria serpyllifolia - Gewone zandmuur,38,3,NaN,NaN,NaN,NaN,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.0,Gewone zandmuur


In [7]:
area_cols = [c for c in Data.columns if "Area" in c]

Data[area_cols] = (
    Data[area_cols]
    .astype(str)                                  
    .apply(lambda col: col.str.strip().str[-1])   
    .replace('n', np.nan)                         
    .replace('', np.nan)                          
    .apply(pd.to_numeric, errors='coerce')        
)

In [8]:
Data.head()

,Species,Name,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,...,Area 93,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102
37,Hogere plant,Achillea millefolium - Duizendblad,8.0,NaN,2.0,8.0,8.0,2.0,8.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
38,Hogere plant,Aira caryophyllea - Zilverhaver,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
39,Hogere plant,Aira praecox - Vroege haver,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
40,Hogere plant,Aphanes australis - Kleine leeuwenklauw,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
41,Hogere plant,Arenaria serpyllifolia - Gewone zandmuur,8.0,3.0,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


In [9]:
df
df=df.drop(columns=["Area 1"], errors="ignore")
df=df.drop(columns=["Name"], errors="ignore")
for i in range(1, df.shape[1]):
    df.columns.values[i] = f"Area {i-1}"
df=df.drop(columns=["Area 0"], errors="ignore")
df



,Species,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,Area 9,...,Area 93,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102
0,Plot,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,NaN,NaN
1,Plot observation,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,NaN,NaN
2,Opnamenummer,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,100,NaN,NaN
3,Associa (1),r14RG04,r16RG20,r16RG20,r32RG01,r32RG01,r32RG01,r16RG20,r16RG20,r16BB01,...,r18AB01,r32RG02,r08RG12,r08BA02,r16RG10,r08RG13,r42RG05,r40AA02,NaN,NaN
4,Bedekkingsschaal,"Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)",...,"Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)","Braun/Blanquet (B,D&S)",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Hogere plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,1097.0,Kluwenzuring
426,Hogere plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,684.0,Zeegroene rus
427,Hogere plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,804.0,Hengel
428,Hogere plant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,2105.0,Pontische rododendron


In [10]:
a=df[6:8]

In [11]:
a=pd.DataFrame(a)
a.shape
Data.shape

(393, 104)

In [12]:
a.insert(1, "New_Column", np.nan)

In [13]:
a.rename(columns={"New_Column": "Name"}, inplace=True)

In [14]:
print(a.shape)
print(type(a))
print(Data.shape)
print(type(Data))

(2, 104)
<class 'pandas.core.frame.DataFrame'>
(393, 104)
<class 'pandas.core.frame.DataFrame'>


In [15]:
df_merged = pd.concat([a, Data], axis=0, ignore_index=True)
df_merged

,Species,Name,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,...,Area 93,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102
0,X-Coordinaat (m),NaN,173806,173805,174027,173696,174118,174086,174290,174243,...,174621.001855,173447.512243,173934,173941,174110,174145,173453.601056,173829.406386,NaN,NaN
1,Y-Coordinaat (m),NaN,444352,444354,444013,443860,444036,444027,444152,444215,...,444090.315717,443732.591499,444180,444215,444269,444302,443819.951806,444209.092018,NaN,NaN
2,Hogere plant,Achillea millefolium - Duizendblad,8.0,NaN,2.0,8.0,8.0,2.0,8.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,Hogere plant,Aira caryophyllea - Zilverhaver,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,Hogere plant,Aira praecox - Vroege haver,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,Hogere plant,Rumex conglomeratus - Kluwenzuring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN
391,Hogere plant,Juncus inflexus - Zeegroene rus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN
392,Hogere plant,Melampyrum pratense - Hengel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN
393,Hogere plant,Rhododendron ponticum - Pontische rododendron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,NaN


In [16]:
df_merged.to_csv("merged_dataset.csv", index=False, sep=";")  

In [17]:
df_merged

,Species,Name,Area 1,Area 2,Area 3,Area 4,Area 5,Area 6,Area 7,Area 8,...,Area 93,Area 94,Area 95,Area 96,Area 97,Area 98,Area 99,Area 100,Area 101,Area 102
0,X-Coordinaat (m),NaN,173806,173805,174027,173696,174118,174086,174290,174243,...,174621.001855,173447.512243,173934,173941,174110,174145,173453.601056,173829.406386,NaN,NaN
1,Y-Coordinaat (m),NaN,444352,444354,444013,443860,444036,444027,444152,444215,...,444090.315717,443732.591499,444180,444215,444269,444302,443819.951806,444209.092018,NaN,NaN
2,Hogere plant,Achillea millefolium - Duizendblad,8.0,NaN,2.0,8.0,8.0,2.0,8.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,Hogere plant,Aira caryophyllea - Zilverhaver,8.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,Hogere plant,Aira praecox - Vroege haver,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,Hogere plant,Rumex conglomeratus - Kluwenzuring,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN
391,Hogere plant,Juncus inflexus - Zeegroene rus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN
392,Hogere plant,Melampyrum pratense - Hengel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,NaN
393,Hogere plant,Rhododendron ponticum - Pontische rododendron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,NaN
